In [4]:
#THaW Project
#This program is meant to establish the accuracy of predicting device name from the data collected by MSU
#6/12/2019
#Code Written By: Manzi Bryan with a lot of help from https://www.kaggle.com/nageshnaik/iris-dataset-classfication-using-naive-bayes
# This was used for the final submission
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import *
import sklearn
from sklearn import model_selection
from sklearn.model_selection import *
import pylab as pl
import pandas as pd
import os
import math
from path import Path
import warnings
import pickle
import operator
import time
import seaborn as sns

# Newer Sklearn

table = pd.DataFrame(index = [0], columns = ['Device', '# train', '# test', 'Accuracy', 'Precision_micro', 
                                                      'Precision_macro', 'Precision_weighted', 'Recall_micro', 
                                                      'Recall_macro','Recall_weighted','F1_micro', 'F1_macro', 
                                                      'F1_weighted', 'Avg Conf', 'Std Conf'])

# This method is meant to build a Random Forest Classifier given a dataframe with devices as targets
# The dontTrainOn parameter is meant to tell the classifier to leave one tenth of one device out of training

def classify(df, functionName, function, dontTrainOn, k):
    save = df[df.Device == dontTrainOn]
    numReadings = len(save)
    
    df = df[df.Device != dontTrainOn] # Drop everything we do not want to train on
    ratio = 10
    ninety = [df, save[save.index % ratio != k]]
    df = pd.concat(ninety) # Concatenate one tenth of readings on the dataframe
    devices = df['Device'].unique()
    i = 0
    df.Device.replace(devices, range(0, len(devices)), inplace=True)
        
    Y = df['Device'].tolist()
    
#     print("There are " + str(len(devices)) + " devices")
    copydf = df.copy()
    
    #Remove labelling columns from the index
    copydf = copydf.drop(columns= ['MSU name','Device','Model','App Running','Distance'])
    X = copydf.values
    
    #Initialize GNB
    clf = function
    
    
    #One tenth of the data as test
    validation_size = 0.1
    
    seed = 7
    
    X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, Y, 
                                                    test_size=validation_size, random_state=seed)
    
    scoring = {'acc': 'accuracy',
           'prec_macro': 'precision_macro',
           'rec_micro': 'recall_macro'}
    
    #Fitting the training set
    clf.fit(X_train, Y_train) 
    
    
    numFeatures = len(X_validation[0])#includes subRegion as a feature
    
   
    #Model Performance
    #setting performance parameters
    kfold = model_selection.StratifiedKFold(n_splits=11, random_state=seed) #same number of samples from each 

    #calling the cross validation function
    
    cv_results = cross_validate(clf, X_train, Y_train, cv=kfold, scoring=scoring, return_train_score=True)
    
    
    
    
    save = save[save.index % ratio == k]
    makePrediction(clf, save, devices, numReadings, dontTrainOn)

def looper(df):
    
    functions = {}
#     functions['RandomForest d=14 n_est=25 '] = RandomForestClassifier(n_estimators=14, max_depth=25)
    functions['RandomForest d=22 n_est=55'] = RandomForestClassifier(n_estimators=22, max_depth=55)
    
    devices = df['Device'].unique()
    
    for functionName in functions:
        df.drop(df.columns[df.columns.str.contains('Unnam',case = False)],axis = 1, inplace = True)#Remove unnecessary columns from the index
#         df.drop(df.columns[df.columns.str.contains('Bursts',case = False)],axis = 1, inplace = True)#Remove unnecessary columns from 
        for device in devices:
            
            k = 0
            for k in range(10):
                print(device + str(k))
                classify(df, functionName, functions[functionName], device, k)
    return devices
    

def getNumerical(devices, y):
    i = 0
    
    for device in devices:
        if device == y[0]:
            return [i] * len(y)
        i += 1

def makePrediction(clf, unseen, devices, numReadings, deviceName):
    unseen = unseen.reindex(sorted(unseen.columns), axis=1)
    
    unseen.drop(unseen.columns[unseen.columns.str.contains('Unname',case = False)],axis = 1, inplace = True)
    
    y = unseen['Device'].tolist() #unseen should only contain one device, hence y is a list of the same device repeated
    
    
    unseen = unseen.drop(columns= ['MSU name','Device','Model','App Running','Distance'])
    
    predictions = clf.predict(unseen)
    
    predictions_proba = clf.predict_proba(unseen)
    i = 0
    other = 0
    # Building Prediction table one prediction at a time
    for i in range(0, len(predictions_proba)):
        confidence = max(predictions_proba[i])
        addToTable(devices, y, predictions, numReadings, deviceName, confidence)
    
    
# This method is meant to help build the results table
def addToTable(devices, y, predictions, numReadings, deviceName, confidence):
    global table
    smallTable = pd.DataFrame(index = [0], columns = ['Device', '# train', '# test', 'Accuracy', 'Precision_micro', 
                                                      'Precision_macro', 'Precision_weighted', 'Recall_micro', 
                                                      'Recall_macro','Recall_weighted','F1_micro', 'F1_macro', 
                                                      'F1_weighted', 'Avg Conf', 'Std Conf'])
    
    correctDevice = getNumerical(devices, y)
    
    smallTable['Device'][0] = deviceName
    
    smallTable['# train'][0] = numReadings - len(predictions)
    
    smallTable['# test'][0] = len(predictions)
    
    smallTable['Accuracy'][0] = accuracy_score(correctDevice, predictions)
    
    smallTable['Precision_micro'][0] = precision_score(correctDevice, predictions, average='micro')    
    smallTable['Precision_macro'][0] = precision_score(correctDevice, predictions, average='macro')
    smallTable['Precision_weighted'][0] = precision_score(correctDevice, predictions, average='weighted')
    
    smallTable['Recall_micro'][0] = recall_score(correctDevice, predictions, average='micro')
    smallTable['Recall_macro'][0] = recall_score(correctDevice, predictions, average='macro')
    smallTable['Recall_weighted'][0] = recall_score(correctDevice, predictions, average='weighted')
    
    smallTable['F1_micro'][0] = f1_score(correctDevice, predictions, average='micro')
    smallTable['F1_macro'][0] = f1_score(correctDevice, predictions, average='macro')
    smallTable['F1_weighted'][0] = f1_score(correctDevice, predictions, average='weighted')
    
    smallTable['Avg Conf'][0] = confidence
    
    
    
    table = table.append(smallTable)
    
    
if __name__ == "__main__":
    
    path=r'C:\Users\f0031pt\ReplayFeatures\train.xlsx'
    
    print("About to read...")
    df = pd.read_excel(path) #Comment this out to run much faster
    print("...read")
    
    df = df.reindex(sorted(df.columns), axis=1)
    df.reindex(range(len(df.Bursts)))
    looper(df) # comment this out to run faster
    
    
    table.to_excel('table.xlsx')
    
    print("Done")
    

    

About to read...
...read
iWatch0


C:\Users\f0031pt\AppData\Local\Continuum\miniconda3\envs\ml\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\f0031pt\AppData\Local\Continuum\miniconda3\envs\ml\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


iWatch1
iWatch2
iWatch3
iWatch4
iWatch5
iWatch6
iWatch7
iWatch8
iWatch9
Bodimetrics Performance monitor0
Bodimetrics Performance monitor1
Bodimetrics Performance monitor2


C:\Users\f0031pt\AppData\Local\Continuum\miniconda3\envs\ml\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\f0031pt\AppData\Local\Continuum\miniconda3\envs\ml\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Bodimetrics Performance monitor3
Bodimetrics Performance monitor4
Bodimetrics Performance monitor5
Bodimetrics Performance monitor6
Bodimetrics Performance monitor7
Bodimetrics Performance monitor8


...

Pyle Health 28


C:\Users\f0031pt\AppData\Local\Continuum\miniconda3\envs\ml\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\f0031pt\AppData\Local\Continuum\miniconda3\envs\ml\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Pyle Health 29


C:\Users\f0031pt\AppData\Local\Continuum\miniconda3\envs\ml\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\f0031pt\AppData\Local\Continuum\miniconda3\envs\ml\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Done


SyntaxError: invalid syntax (<ipython-input-5-e43a5b113cad>, line 12)